In [88]:
! pip install autogen-core
! pip install autogen-agentchat
! pip install autogen-ext

In [89]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from kaggle_secrets import UserSecretsClient
import os

user_secrets = UserSecretsClient()

secret_value_0 = user_secrets.get_secret("GEMINI_API_KEY")
secret_value_1 = user_secrets.get_secret("OPENAI_API_KEY")
os.environ['GEMINI_API_KEY'] = secret_value_0
os.environ['OPENAI_API_KEY'] = secret_value_1

In [90]:
@dataclass
class Message:
    content: str


class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("simple_agent")   # agent type name

    @message_handler
    async def on_my_message(self, message: Message, ctx: MessageContext) -> Message:
        reply = (
            f"[{self.id.type}-{self.id.key}] Agent Active\n"
            f"Received: '{message.content}'\n"
            "Status: ONLINE\n"
            "Capabilities:\n"
            " - Text Processing\n"
            " - Command Execution\n"
            " - Context Awareness\n"
            "Awaiting next instruction..."
        )
        return Message(content=reply)




runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
        

AgentType(type='simple_agent')

In [91]:
runtime.start()

In [92]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Start the interface. Provide an overview of what you can do."), agent_id)
print(">>>", response.content)

>>> [simple_agent-default] Agent Active
Received: 'Start the interface. Provide an overview of what you can do.'
Status: ONLINE
Capabilities:
 - Text Processing
 - Command Execution
 - Context Awareness
Awaiting next instruction...


In [93]:
await runtime.stop()
await runtime.close()

In [94]:
class MyLLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent("LLMAgent", model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")

        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)

        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")

        return Message(content=reply)

    


runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await MyLLMAgent.register(runtime, "LLMAgent", lambda: MyLLMAgent())

AgentType(type='LLMAgent')

In [95]:
runtime.start()  

# Step 1
print("#"*130)
response1 = await runtime.send_message(Message("Explain how quantum entanglement works, step by step, and give a real-world analogy."), AgentId("LLMAgent", "default"))
print("LLM received user:", response1.content)
print("#"*130)

# Step 2
response2 = await runtime.send_message(Message(response1.content), AgentId("simple_agent", "default"))
print("SimpleAgent responded:", response2.content)
print("#"*130)

# Step 3
response3 = await runtime.send_message(Message(response2.content), AgentId("LLMAgent", "default"))
print("LLM responded to SimpleAgent:", response3.content)
print("#"*130)

##################################################################################################################################
LLMAgent received message: Explain how quantum entanglement works, step by step, and give a real-world analogy.
LLMAgent responded: Quantum entanglement is a fascinating phenomenon in quantum mechanics where pairs or groups of particles become interconnected in such a way that the state of one particle instantly influences the state of another, regardless of the distance separating them. Here's a step-by-step explanation of how it works, followed by a real-world analogy:

### Step-by-Step Explanation

1. **Quantum States**: Particles, like electrons or photons, have quantum states. These can represent various properties, such as spin or polarization.

2. **Entanglement Creation**: Two particles can become entangled through interactions, such as when they collide or are produced together in certain processes. This means their quantum states are linked, even 

In [96]:
await runtime.stop()
await runtime.close()

In [97]:
from autogen_core.models import ModelInfo

class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gemini-2.5-flash",api_key=secret_value_0,temperature=1.0,
            model_info=ModelInfo(vision=False, function_calling=False, json_output=False, family="gemini", structured_output=False))
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [98]:
JUDGE = """You are an expert judge for Rock-Paper-Scissors-Lizard-Spock.

Rules:
Scissors cuts Paper
Paper covers Rock
Rock crushes Lizard
Lizard poisons Spock
Spock smashes Scissors
Scissors decapitates Lizard
Lizard eats Paper
Paper disproves Spock
Spock vaporizes Rock
Rock crushes Scissors

The players have made these choices:
"""

class  RPSLSAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini", temperature=0.7)
        self._delegate = AssistantAgent(name, model_client=model_client)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        instruction = ("You are playing Rock-Paper-Scissors-Lizard-Spock.\n" "Respond with exactlyily ONE of these words (exact spelling, lowercase):\n"
            "rock, paper, scissors, lizard, spock")
        msg = Message(content=instruction)
        
        p1 = AgentId("player1", "default")
        p2 = AgentId("player2", "default")
        
        resp1 = await self.send_message(msg, p1)
        resp2 = await self.send_message(msg, p2)
        
        result = f"Player 1: {resp1.content.strip().lower()}\nPlayer 2: {resp2.content.strip().lower()}\n"
        judgment_prompt = f"{JUDGE}{result}Who wins? Say 'Tie!' if it's a tie, otherwise declare the winner and explain in 1 short sentence."

        judge_msg = TextMessage(content=judgment_prompt, source="user")
        judgment = await self._delegate.on_messages([judge_msg], ctx.cancellation_token)

        return Message(content=result + "\n" + judgment.chat_message.content)



In [99]:
runtime = SingleThreadedAgentRuntime()

await RPSLSAgent.register(runtime, "rpsls", lambda: RPSLSAgent("rpsls_judge"))
await Player1Agent.register(runtime, "player1", lambda: Player1Agent("player1"))
await Player2Agent.register(runtime, "player2", lambda: Player2Agent("player2"))

AgentType(type='player2')

In [100]:
runtime.start()

In [101]:
agent_id = AgentId("rpsls", "default")
response = await runtime.send_message(Message(content="go"), agent_id)
print(response.content)

Player 1: lizard
Player 2: rock

Player 2 wins, as Rock crushes Lizard. TERMINATE


In [102]:
await runtime.stop()
await runtime.close()